In [7]:
from astroquery.alma import Alma
alma=Alma()

from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.table import QTable
from astropy.time import Time
from astropy.io import fits

from astroplan import Observer

In [33]:
class QuasarPol:
    
    def __init__(self, source, sci_obs, table_length):
        '''
        constructor of the class
        '''
        self.science = sci_obs
        self.len = table_length
        self.source = source
        
        self.ObsCore_table = alma.query(payload=dict(source_name_alma=self.source),
                                        science=self.science,
                                        maxrec=self.len
                                       )
        
        self.ALMA_table = alma.query(payload=dict(source_name_alma=self.source),
                                     science=self.science,
                                     legacy_columns=True, 
                                     maxrec=self.len
                                    )
    
    def __del__(self):
        '''
        Destrucror of th class
        '''
    
    def Obstable():
        '''
        Creating this for the purpose of not to creating tables in constructor
        '''
        pass
    
    def almatable():
        '''
        Creating this for the purpose of not to creating tables in constructor
        '''
        pass
    
    def get_ParaAngle(self):
        
        Obs_ids = self.ObsCore_table['obs_id']
        Uids = self.ObsCore_table['member_ous_uid']
        Init_PA = []
        End_PA = []
        
        for i in range(len(Obs_ids)):
            
            ALMA = Observer.at_site("ALMA")
            
            # Get source coordinate
            Ra = self.ALMA_table['RA'][i]
            Dec = self.ALMA_table['Dec'][i]
            target_coord = SkyCoord(ra=Ra*u.deg, dec=Dec*u.deg)
            
            # Get date
            date = self.ALMA_table['Observation date'][i]
            [day, month, year] = date.split('-')
            obs_date = year + '-' + month + '-' + day
            
            # Get observation time information
            start_time = self.ObsCore_table['t_min'][i]
            duration_time = self.ObsCore_table['t_exptime'][i]
            end_time = start_time + duration_time
            
            # Transform into the format we can understand (UTC)
            hours = int(start_time / 3600)
            remaining_seconds = start_time % 3600
            minutes = int(remaining_seconds / 60)
            seconds = remaining_seconds - minutes * 60
            
            obs_start_time = str(hours)+':'+str(minutes)+':'+str(seconds)
            
            # combine time and date
            obs_init_Datetime = Time(obs_date + ' ' + obs_start_time)
            
            # Initial Parallactic Angle calculation and create list
            init_PA = Angle(ALMA.parallactic_angle(obs_init_Datetime, target_coord), u.deg)
            Init_PA.append(init_PA)
            
            # Final Parallactic Angle Part
            hours = int(end_time / 3600)
            remaining_seconds = end_time % 3600
            minutes = int(remaining_seconds / 60)
            seconds = remaining_seconds - minutes * 60
            
            obs_end_time = str(hours)+':'+str(minutes)+':'+str(seconds)
            obs_end_Datetime = Time(obs_date + ' ' + obs_end_time)
            
            end_PA = Angle(ALMA.parallactic_angle(obs_end_Datetime, target_coord), u.deg)
            End_PA.append(end_PA)
        
        t = QTable([Obs_ids, Uids, Init_PA, End_PA], 
                   names=('obs_id','member_ous_uid', 'Initial Parallactic Angle', 'Final Parallactic Angle'))

        return t
    
    
    def download(self, obs_id):
        
        pass

In [14]:
source = 'J1924-2914'
baobab = 'Hauyu Baobob Liu'

In [34]:
data = QuasarPol('J1924-2914', False, 100)

/home/Peter/Softwares/anaconda3/envs/DataQuery/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",
/home/Peter/Softwares/anaconda3/envs/DataQuery/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [35]:
data.get_ParaAngle()

obs_id,member_ous_uid,Initial Parallactic Angle,Final Parallactic Angle
,,deg,deg
str64,str64,float64,float64
uid://A001/X128c/X12.source.J1924-2914.spw.25,uid://A001/X128c/X12,-129.1907819575085,-127.57602702478374
uid://A001/X128c/X12.source.J1924-2914.spw.27,uid://A001/X128c/X12,-129.1907819575085,-127.57602702478374
uid://A001/X128c/X12.source.J1924-2914.spw.31,uid://A001/X128c/X12,-129.1907819575085,-127.57602702478374
uid://A001/X128c/X12.source.J1924-2914.spw.29,uid://A001/X128c/X12,-129.1907819575085,-127.57602702478374
uid://A001/X1465/Xe9d.source.J1924-2914.spw.18,uid://A001/X1465/Xe9d,-159.53385114780122,-151.8291110786885
uid://A001/X1465/Xe9d.source.J1924-2914.spw.20,uid://A001/X1465/Xe9d,-159.53385114780122,-151.8291110786885
...,...,...,...
uid://A001/X1469/X198.source.J1924-2914.spw.29,uid://A001/X1469/X198,-95.72784631611671,-95.20681077392165
